# Preprocessing

In [1]:
import sklearn
import pandas as pd
import sklearn.metrics as metrics
import numpy as np
from sklearn import metrics
from sklearn import model_selection
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import confusion_matrix
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.ensemble import RandomForestClassifier

In [2]:
x=pd.read_csv("updatedPK.csv")
del x ['Cmpd Name']
x = x.dropna(axis = 0, how='any')
x = x.sort_values(['AUC'], ascending = False)
x.reset_index(drop = True, inplace = True)
smiles = x['mol']
del x ['mol']
x["AUC1"]=" "

for i in range(len(x)): 
    if x['AUC'][i]<= 1000:
        x.iat[i, 207] = 0
    else:
        x.iat[i, 207] = 1 
x['AUC1'].value_counts()

1    96
0    94
Name: AUC1, dtype: int64

In [3]:
del x['AUC']
y = x['AUC1']
del x['AUC1']

In [4]:
from sklearn.feature_selection import VarianceThreshold
def variance_threshold_selector(x, threshold = 0.2):
    selector = VarianceThreshold(threshold)
    selector.fit(x)
    return x[x.columns[selector.get_support(indices=True)]] 

x = variance_threshold_selector(x, 0.2)
cor_matrix = x.corr().abs()
upper_tri = cor_matrix.where(np.triu(np.ones(cor_matrix.shape), k=1).astype(np.bool))
to_drop = [column for column in upper_tri.columns if any(upper_tri[column] >= 0.9)]
print(); print(to_drop)


['ast_violation_ext', 'a_base', 'a_count', 'a_donacc', 'a_heavy', 'a_hyd', 'a_IC', 'a_nBr', 'a_nC', 'a_nF', 'a_nH', 'a_nI', 'balabanJ', 'BCUT_PEOE_3', 'BCUT_SLOGP_0', 'BCUT_SLOGP_3', 'BCUT_SMR_3', 'bpol', 'b_1rotR', 'b_count', 'b_double', 'b_heavy', 'b_rotN', 'b_rotR', 'b_single', 'b_triple', 'chi0', 'chi0v', 'chi0_C', 'chi1', 'chi1v', 'chi1v_C', 'chi1_C', 'density', 'diameter', 'GCUT_PEOE_3', 'GCUT_SLOGP_3', 'GCUT_SMR_2', 'GCUT_SMR_3', 'h_logP', 'h_logS', 'h_log_dbo', 'h_mr', 'h_pavgQ', 'Kier1', 'Kier2', 'KierA1', 'KierA2', 'mr', 'nmol', 'PC-', 'PEOE_PC-', 'PEOE_RPC-', 'PEOE_VSA-1', 'PEOE_VSA_FPPOS', 'PEOE_VSA_HYD', 'Q_VSA_FPPOS', 'Q_VSA_HYD', 'Q_VSA_POL', 'Q_VSA_POS', 'Q_VSA_PPOS', 'radius', 'rings', 'SlogP_VSA6', 'SMR', 'TPSA', 'VAdjEq', 'VDistEq', 'VDistMa', 'vdw_vol', 'vsa_base', 'vsa_don', 'vsa_hyd', 'vsa_other', 'vsa_pol', 'Weight', 'weinerPath', 'weinerPol', 'zagreb']


/var/folders/p_/665xkkjx28zdr3jqxqhthkgh0000gn/T/ipykernel_48512/3421802328.py:9: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  upper_tri = cor_matrix.where(np.triu(np.ones(cor_matrix.shape), k=1).astype(np.bool))


In [5]:
x=x.drop(columns=['ast_violation_ext', 'a_base', 'a_count', 'a_donacc', 'a_heavy', 'a_hyd', 'a_IC', 'a_nBr', 'a_nC', 'a_nF', 'a_nH', 'a_nI', 'balabanJ', 'BCUT_PEOE_3', 'BCUT_SLOGP_0', 'BCUT_SLOGP_3', 'BCUT_SMR_3', 'bpol', 'b_1rotR', 'b_count', 'b_double', 'b_heavy', 'b_rotN', 'b_rotR', 'b_single', 'b_triple', 'chi0', 'chi0v', 'chi0_C', 'chi1', 'chi1v', 'chi1v_C', 'chi1_C', 'density', 'diameter', 'GCUT_PEOE_3', 'GCUT_SLOGP_3', 'GCUT_SMR_2', 'GCUT_SMR_3', 'h_logP', 'h_logS', 'h_log_dbo', 'h_mr', 'h_pavgQ', 'Kier1', 'Kier2', 'KierA1', 'KierA2', 'mr', 'nmol', 'PC-', 'PEOE_PC-', 'PEOE_RPC-', 'PEOE_VSA-1', 'PEOE_VSA_FPPOS', 'PEOE_VSA_HYD', 'Q_VSA_FPPOS', 'Q_VSA_HYD', 'Q_VSA_POL', 'Q_VSA_POS', 'Q_VSA_PPOS', 'radius', 'rings', 'SlogP_VSA6', 'SMR', 'TPSA', 'VAdjEq', 'VDistEq', 'VDistMa', 'vdw_vol', 'vsa_base', 'vsa_don', 'vsa_hyd', 'vsa_other', 'vsa_pol', 'Weight', 'weinerPath', 'weinerPol', 'zagreb'])

In [6]:
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
scaler = preprocessing.StandardScaler().fit(x)
X = scaler.transform(x) 
x1 = pd.DataFrame(X) 
x1.columns = x.columns

# Fingerprints

In [7]:
from rdkit import Chem
from rdkit.Chem.AllChem import GetMorganFingerprintAsBitVect
from rdkit.DataStructs.cDataStructs import ConvertToNumpyArray

smiles = pd.DataFrame(smiles)
smiles = smiles.dropna(axis = 0, how='any')

fp = []
for i in smiles['mol']:
    w = Chem.MolFromSmiles(i)
    fp.append(w)
    
fp = pd.DataFrame(fp)
fp.columns = ['Fingerprint']

x_wfp = pd.concat([x1, fp], axis=1)
x_wfp1 = x_wfp.dropna(axis=0, how='any') 
x_wfp2 = x_wfp1.reset_index(drop=True)

fp1 = []

for i in x_wfp2['Fingerprint']:
    fingerprint = GetMorganFingerprintAsBitVect(i, radius=2, nBits=4096)
    fp1.append(fingerprint)
    
fp_final = []
for i in fp1:
    fp_array = np.zeros((1, ))
    ConvertToNumpyArray(i, fp_array)
    fp_final.append(fp_array)
    
x_fp = pd.DataFrame(fp_final)
x_fp.reset_index(drop=True, inplace=True)

del x_wfp2['Fingerprint']

x1 = pd.concat([x_wfp2, x_fp], axis=1)
combined = pd.concat([y,smiles,x1], axis = 1)
combined

,AUC1,mol,apol,ast_violation,a_acc,a_acid,a_aro,a_don,a_nCl,a_nN,...,4086,4087,4088,4089,4090,4091,4092,4093,4094,4095
0,1,Clc1c2sc(C(=O)NNC(=O)c3oc([N+](=O)[O-])cc3)cc2...,-0.348564,0.160043,-0.889080,-0.175133,0.270776,0.063452,1.498092,-0.497626,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,Clc1c(C(=O)NNC(=O)c2oc([N+](=O)[O-])cc2)sc2c1c...,-0.440130,0.160043,-0.889080,-0.175133,0.270776,0.063452,3.545843,-0.497626,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,Clc1c(C(=O)NNC(=O)c2oc([N+](=O)[O-])cc2)[nH]c2...,-0.372807,0.160043,-0.065055,-0.175133,0.270776,0.816946,1.498092,-0.010260,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,Clc1c(C(=O)NNC(=O)c2oc([N+](=O)[O-])cc2)[nH]c2...,-0.431508,0.160043,-0.889080,-0.175133,0.270776,0.816946,1.498092,-0.010260,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,S(=O)(=O)(Nc1cc2c(C)n[nH]c2cc1)CCCCF,-0.712986,-1.222150,0.758971,-0.175133,-0.805530,0.816946,-0.549660,-0.497626,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185,0,S(=O)(=O)(Nc1cc2c(C)nsc2cc1)CCCCC,-0.395575,-1.222150,-0.065055,-0.175133,-0.805530,-0.690042,-0.549660,-0.984991,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
186,0,Clc1ccc(CCC(=O)c2c(O)cc(O)cc2[O-])cc1,-0.739348,-1.222150,-0.065055,0.429873,-0.159746,0.063452,1.498092,-1.959722,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
187,0,O=C(Nc1c2c(cnnc2)ccc1)c1nccnc1,-1.076506,-1.222150,1.582997,-0.175133,0.701298,-0.690042,-0.549660,0.477105,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
188,0,O=C(Nc1c2c(ncc1)cccc2)c1nccnc1,-0.979393,-1.222150,0.758971,-0.175133,0.701298,-0.690042,-0.549660,-0.010260,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Stratified Sampling

In [8]:
x1_train, x1_test = train_test_split(combined, stratify = combined['AUC1'], test_size = 0.40, random_state = 1)
x1_cv, x1_et = train_test_split(x1_test, stratify = x1_test['AUC1'], test_size = 0.5, random_state = 1)

In [9]:
x1_train.to_csv('pk_60_40_training.csv', index=False)
x1_cv.to_csv('pk_60_40_cv.csv', index=False)
x1_et.to_csv('pk_60_40_et.csv', index=False)

## Seed=42

In [10]:
x1_train, x1_test = train_test_split(combined, stratify = combined['AUC1'], test_size = 0.40, random_state = 42)
x1_cv, x1_et = train_test_split(x1_test, stratify = x1_test['AUC1'], test_size = 0.5, random_state = 42)

In [11]:
x1_train.to_csv('pk_60_40_training_seed42.csv', index=False)
x1_cv.to_csv('pk_60_40_cv_seed42.csv', index=False)
x1_et.to_csv('pk_60_40_et_seed42.csv', index=False)